# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import requests
import re
from bs4 import BeautifulSoup

### 先搜尋全部的電影代號(ID)資訊

In [3]:
# 查看目前上映那些電影，並擷取出其ID資訊
url = 'https://movies.yahoo.com.tw/'
resp = requests.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
html = soup.find("select", attrs={'name':'movie_id'})
movie_item = html.find_all("option", attrs={'data-name':re.compile('.*')})

for p in movie_item:
    print("Movie: %s, ID: %s" % (p["data-name"], p["value"]))

Movie: 鬼弒訊, ID: 11083
Movie: 迷走廣州, ID: 11060
Movie: 無間行者之生死潛行, ID: 11051
Movie: 未來的我們, ID: 11046
Movie: 卡卡囌卡卡衝, ID: 11043
Movie: 他們的自由年代, ID: 11037
Movie: MISS, ID: 11036
Movie: 十二夜2：回到第零天, ID: 11035
Movie: 尋找小魔女Doremi, ID: 11027
Movie: 外星異種, ID: 11025
Movie: 緝毒風暴, ID: 11023
Movie: 說不出的告別, ID: 11019
Movie: 追殺胖老爹, ID: 11015
Movie: 鹿兒流浪記, ID: 11012
Movie: 謎樣事件簿, ID: 11011
Movie: 慾望摩天輪, ID: 11010
Movie: 古魯家族：新石代, ID: 10958
Movie: 讓我聽見你的聲音, ID: 11040
Movie: 盜墓同盟, ID: 11031
Movie: 槍手養成班, ID: 11009
Movie: 愛在末路之境, ID: 11008
Movie: 婊兄弟上路, ID: 11003
Movie: 瘋狂理髮師, ID: 10998
Movie: 你是豬, ID: 10995
Movie: 消失的星期三, ID: 10949
Movie: 同學麥娜絲, ID: 10935
Movie: 惡童當街 經典重映, ID: 10932
Movie: 名偵探柯南：紅之校外旅行 鮮紅篇&戀紅篇, ID: 10887
Movie: 逃, ID: 10565
Movie: 海邊的異邦人, ID: 11014
Movie: 弒膽遊戲, ID: 10988
Movie: 換人殺砍砍, ID: 10933
Movie: 祕密：夢想的力量, ID: 10566
Movie: 愛麗絲與夢幻島, ID: 11018
Movie: 家在蘭若寺, ID: 11034
Movie: 5x1, ID: 11032
Movie: GIVEN~被贈與的未來劇場版, ID: 11022
Movie: 綠洲：數位經典版, ID: 11013
Movie: 藥命時空, ID: 11004
Movie: 孤味, ID: 1

### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [4]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = 10169

In [5]:
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
payload = {'movie_id':str(movie_id)}

# call api 會發現權限不足所以需按照 Network中的request，模擬一個header
# 模擬一個header
headers = {
    'authority': 'movies.yahoo.com.tw',
    'method': 'GET',
    'path': '/api/v1/areas_by_movie_theater?movie_id=' + str(movie_id),
    'scheme': 'https',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'rxx=9s3x2fws06.1g16irnc&v=1; _ga=GA1.3.2056742944.1551651301; GUC=AQEBAQFczFpdm0IfmwSB&s=AQAAACoo4N5D&g=XMsVBw; BX=4hkdk1decm57t&b=3&s=mr; _ga=GA1.4.2056742944.1551651301; nexagesuid=82843256dd234e8e91aa73f2062f8218; browsed_movie=eyJpdiI6IlJXWWtiSWJaZlNGK2MxQnhscnVUYWc9PSIsInZhbHVlIjoiMXRhMmVHRXRIeUNjc1RBWDJzdGYwbnlIQURmWGsrcjJSMzhkbkcraDNJVUNIZEZsbzU3amlFcVZ1NzlmazJrTGpoMjVrbHk1YmpoRENXaHZTOUw1TmI2ZTZVWHdOejZQZm16RmVuMWlHTTJLaTZLVFZZVkFOMDlTd1wvSGltcytJIiwibWFjIjoiZWQ2ZjA4MmVjZmZlYjlmNjJmYmY2NGMyMDI0Njc0NWViYjVkOWE2NDg0N2RhODMxZjBjZDhiMmJhZTc2MDZhYiJ9; avi=eyJpdiI6Im1NeWFJRlVRWDR1endEcGRGUGJUbVE9PSIsInZhbHVlIjoickRpU3JuUytmcGl6cjF5OW0rNU9iZz09IiwibWFjIjoiY2VmY2NkNzZmM2NhNjY5YzlkOTcyNjE5OGEyMzU0NWYxOTdmMDRkMDY3OWNmMmZjOTMxYjc5MjI5N2Q5NGE5MiJ9; cmp=t=1559391030&j=0; _gid=GA1.4.779543841.1559391031; XSRF-TOKEN=eyJpdiI6IkhpS2hGcDRQaHlmWUJmaHdSS2Q2bHc9PSIsInZhbHVlIjoiOUVoNFk4OHI1UUZmUWRtYXhza0MyWjJSTlhlZ3RnT0VGeVJPN2JuczVRMGRFdWt2OUlsamVKeHRobFwvcHBGM0dhU3VyMXNGTHlsb2dVM2l0U1hpUGxBPT0iLCJtYWMiOiJkZWU4YzJhNjAxMTY3MzE4Y2ExNWIxYmE1ZjE1YWZlZTlhOTcyYjc4M2RlZGY4ZWNjZDYyMTA2NGYwZGViMzc2In0%3D; m_s=eyJpdiI6InpsZHZ2Tk1BZ0dxaHhETml1RjBnUXc9PSIsInZhbHVlIjoiSkNGeHUranRoXC85bDFiaDhySTJqNkJRcWdjWUxjeVRJSHVYZ1wvd2d4bWJZUTUrSHVDM0lUcW5KNHdETFZ4T1lieU81OUhzc1VoUXhZcWk0UDZSQXVFdz09IiwibWFjIjoiYmJkMDJkMDhlODIzMzcyMWY4M2NmYWNjNGVlOWRjMDIwZmVmNzAyMjE3Yzg3ZGY3ODBkZWEzZTI4MTI5ZWNmOSJ9; _gat=1; nexagesd=10',
    'dnt': '1',
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
    'referer': 'https://movies.yahoo.com.tw/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}
    
resp = requests.get(url, params=payload, headers=headers)
# print(resp.json())  # 若有需要，列印出json原始碼

# 這裡回傳的格式是JSON格式的資料，要解析JSON擷取資料
for p in resp.json():
    print('放映地區: {0}, 代號(area_id): {1}'.format(p['title'], p['area_id']))

### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [6]:
# 指定放映地區
area_id = 28

In [7]:
# 向網站發送請求
url = 'https://movies.yahoo.com.tw/movietime_result.html'
payload = {'movie_id':str(movie_id), 'area_id':str(area_id)}
resp = requests.get(url, params=payload)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
movie_date = soup.find_all("label", attrs={'for':re.compile("date_[\d]")})

# 列印播放日期
for date in movie_date:
    print("%s %s" % (date.p.string, date.h3.string))

十二月 3
十二月 4
十二月 5
十二月 6
十二月 7


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [8]:
# 選定要觀看的日期
date = "2019-08-21"

In [9]:
# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {'movie_id':str(movie_id),
           'date':date,
           'area_id':str(area_id),
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}

resp = requests.get(url, params=payload)
#print(resp.json()['view'])  # 若有需要，列印出json原始碼

soup = BeautifulSoup(resp.json()['view'], 'lxml')
html = soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")})

In [10]:

for theater in html:
    print("----------------------------------------------------------------------")
    print("電影院:  %s" % theater['data-theater_name'])
    movie_info = theater.find_all("li", attrs={"class":"taps"})
    taps = []
    for info in movie_info:
        movie_type = info.find_all("span")
        tap = []
        for mtype in movie_type:
            tap.append(mtype.text)
        taps.append(' '.join(tap))
    movie_info = theater.find_all("div", attrs={"class":"input_picker jq_input_picker"})
    for info in range(len(movie_info)):
        print("放映類型:  %s" % taps[info])
        schedule_list = movie_info[info].find_all("input", attrs={"class":"gabtn"})
        for schedule in schedule_list:
            print(schedule['value'])